In [11]:
import os
import json
import pandas as pd

def extract_and_transform_data(base_path="pulse/data/aggregated/transaction/country/india/state/"):
    data_list = []

    # Traversing through all states
    for state_folder in os.listdir(base_path):
        state_path = os.path.join(base_path, state_folder)

        if os.path.isdir(state_path):  # Check if it is a directory (state)
            for year in os.listdir(state_path):
                year_path = os.path.join(state_path, year)
                
                if os.path.isdir(year_path):  # Check if it is a directory (year)
                    for quarter_file in os.listdir(year_path):
                        file_path = os.path.join(year_path, quarter_file)
                        
                        if file_path.endswith('.json'):
                            with open(file_path, 'r') as f:
                                data = json.load(f)

                            for transaction in data.get('data', {}).get('transactionData', []):
                                record = {
                                    'state': state_folder,
                                    'year': int(year),
                                    'quarter': int(quarter_file.replace('.json', '')),
                                    'name': transaction['name'],
                                    'type': transaction['paymentInstruments'][0]['type'],
                                    'count': transaction['paymentInstruments'][0]['count'],
                                    'amount': transaction['paymentInstruments'][0]['amount']
                                }
                                data_list.append(record)

    # Creating a DataFrame
    df = pd.DataFrame(data_list)
    return df

# Extract and transform the data
df = extract_and_transform_data()
print(df.head())


                       state  year  quarter                      name   type  \
0  andaman-&-nicobar-islands  2018        1  Recharge & bill payments  TOTAL   
1  andaman-&-nicobar-islands  2018        1     Peer-to-peer payments  TOTAL   
2  andaman-&-nicobar-islands  2018        1         Merchant payments  TOTAL   
3  andaman-&-nicobar-islands  2018        1        Financial Services  TOTAL   
4  andaman-&-nicobar-islands  2018        1                    Others  TOTAL   

   count        amount  
0   4200  1.845307e+06  
1   1871  1.213866e+07  
2    298  4.525072e+05  
3     33  1.060142e+04  
4    256  1.846899e+05  


In [13]:
import mysql.connector

# Connect to MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root@1234",
    database="phonepe_data"
)
cursor = connection.cursor()

# Creating a table to store data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        state VARCHAR(50),
        year INT,
        quarter INT,
        name VARCHAR(100),
        type VARCHAR(50),
        count BIGINT,
        amount FLOAT
    )
''')

# Inserting data into the table
for _, row in df.iterrows():
    cursor.execute('''
        INSERT INTO transactions (state, year, quarter, name, type, count, amount)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''', tuple(row))

connection.commit()
connection.close()
